In [13]:
import torch
import whisper
from utils import get_device 

In [14]:
device = get_device()
model = whisper.load_model("tiny", device=device)
tokenizer = whisper.tokenizer.get_tokenizer(multilingual=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

Using device: cuda


In [15]:
# Load and process the audio file
FILE_PATH = "../audio/Helen--Bes.m4a"
audio = whisper.load_audio(FILE_PATH)
audio = whisper.pad_or_trim(audio)
log_mel = whisper.log_mel_spectrogram(audio).to(device)

In [16]:
options = whisper.DecodingOptions()
response = whisper.decode(model, log_mel, options)
print("whisper prediction without fine tuning: ", response.text)

whisper prediction without fine tuning:  Hello my name is Beds


In [ ]:
# Preparing input for target for the model to train on and learn
ids = []
ids += [tokenizer.sot]
ids += [tokenizer.language_token]
ids += [tokenizer.transcribe]
ids += [tokenizer.no_timestamps]
ids += tokenizer.encode(" Hello, my name is Bes.")
ids += [tokenizer.eot]

model.train()

train_tokens = torch.tensor(ids, device=device).unsqueeze(0)
mel_unsqueezed = log_mel.unsqueeze(0).to(device)
prediction = model(tokens=train_tokens, mel=mel_unsqueezed)
target = train_tokens[:, 1:].contiguous()  # Skip the start token

print("--- Before training ---")
print("Ids Target: ", target.squeeze().tolist())
print("Ids Pred: ", torch.argmax(prediction, dim=-1).squeeze().tolist())
print("Text target: ", tokenizer.decode(target.squeeze().tolist()))
print("Text pred: ", tokenizer.decode(torch.argmax(prediction, dim=-1).squeeze().tolist()))
loss = criterion(prediction.transpose(1, 2), train_tokens)
print("Loss: ", loss.item())

--- Before training ---
Ids Target:  [50259, 50359, 2425, 11, 452, 1315, 307, 8190, 13, 50257]
Ids Pred:  [50259, 50359, 50363, 452, 452, 1315, 307, 363, 13, 50257, 50257]
Text target:  <|en|><|transcribe|> Hello, my name is Bes.<|endoftext|>
Text pred:  <|en|><|transcribe|><|notimestamps|> my my name is B.<|endoftext|><|endoftext|>
Loss:  12.477005958557129


In [20]:
training_count = 0

In [ ]:
# Training the model - re-run this cell to have it train multiple times
training_count += 1
print(f"---- Training count {training_count} ----")
optimizer.zero_grad()
loss.backward()
optimizer.step()

print("--- After training ---")
model.eval()
prediction = model(tokens=train_tokens, mel=mel_unsqueezed)
prediction = prediction[:, :-1, :].contiguous()  # Remove the last token

print("Ids Target: ", target.squeeze().tolist())
print("Ids Pred: ", torch.argmax(prediction, dim=-1).squeeze().tolist())
print("Text target: ", tokenizer.decode(target.squeeze().tolist()))
print("Text pred: ", tokenizer.decode(torch.argmax(prediction, dim=-1).squeeze().tolist()))

loss = criterion(prediction.transpose(1, 2), target)
print("Loss: ", loss.item())


---- Training count 5 ----
--- After training ---
Ids Target:  [50259, 50359, 2425, 11, 452, 1315, 307, 8190, 13, 50257]
Ids Pred:  [50259, 50359, 50363, 11, 452, 1315, 307, 8190, 13, 50257]
Text target:  <|en|><|transcribe|> Hello, my name is Bes.<|endoftext|>
Text pred:  <|en|><|transcribe|><|notimestamps|>, my name is Bes.<|endoftext|>
Loss:  1.0054512023925781
